In [ ]:
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Dense, Lambda

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
import efficientnet.tfkeras as efn 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers.core import Dense, Dropout, Flatten, Activation
#from tensorflow.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.keras.utils import np_utils
import os
import numpy as np
from tqdm import tqdm 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
train_image_path = './train/'
test_image_path = './test/'
ex_test_image_path = './extest/'


In [ ]:
image_list = []
import os
def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['0']:
                label = 0
            elif nextDir in ['1']:
                label = 1

                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                image = load_img(temp + '/' + file,target_size=(256,256))
                image = img_to_array(image)
                image /= 255
                X.append(image)
                y.append(label)
                image_list.append(file)   
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [ ]:
x_train,y_train = get_data(train_image_path)
x_test,y_test = get_data(test_image_path)
ex_x_test,ex_y_test = get_data(ex_test_image_path)

In [ ]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train, 2)
Y_test = to_categorical(y_test, 2)
ex_Y_test = to_categorical(ex_y_test, 2)

In [ ]:
import efficientnet.tfkeras as efn 
efnmodel = efn.EfficientNetB5(include_top=True, weights=None, input_tensor=None, input_shape=(256,256,3))  # or weights='noisy-student'
efnmodel.load_weights('./weights/efficientnet-b5_weights_tf_dim_ordering_tf_kernels.h5')

x = efnmodel.layers[-2].output
output_layer = Dense(2, activation='sigmoid', name='fc')(x)

# combine the original model with the new output layer
efnmodel2 = Model(inputs=efnmodel.input, outputs=output_layer)

# compile the new model
efnmodel2.compile(optimizer=Adam(lr=0.00008),
                loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = 'efn2.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
hist = efnmodel2.fit(x_train, Y_train, 
                 batch_size=32, 
                 epochs=100, 
                 verbose=1,
                 callbacks=[checkpoint],
                 validation_data = (x_test , Y_test),
                 shuffle =True)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['savefig.dpi'] = 500
plt.rcParams['figure.dpi'] = 500

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('')
plt.ylabel('Loss')
plt.ylim([0.0, 10.01])
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.show()
# summarize history for loss
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1.01])
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.show()

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import efficientnet.tfkeras as efn 
from tensorflow.keras.models import Model, load_model
model_a = load_model('efn2.h5')




from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False, title='', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    plt.rcParams['savefig.dpi'] = 300
    plt.rcParams['figure.dpi'] = 300
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
Y_pred = model_a.predict(x_test)
# Convert predictions classes from one hot vectors to labels: [0 0 1 0 0 ...] --> 2
y_pred = np.argmax(Y_pred, axis=1)
# Convert validation observations from one hot vectors to labels
#y_true = np.argmax(Y_valid, axis=1)
y_true = y_test
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=['Begign','Invasive breast cancer'])

In [ ]:
confusion_mtx = confusion_matrix(y_true, y_pred)
# plot the confusion matrix

con_mat_norm = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]     # 归一化
con_mat_norm = np.around(con_mat_norm, decimals=2)

plot_confusion_matrix(con_mat_norm, ['Benign','Invasive breast cancer'])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print('Classification Report')
print(classification_report(y_true, y_pred))

    
from sklearn.metrics import roc_curve, auc
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(Y_test.ravel(), Y_pred.ravel())

# Compute ROC area
roc_auc = auc(fpr, tpr)
print('ROC area is {0}'.format(roc_auc))